# Most Dangerous Streets - Crash Leaderboard

Compare the most recent month to the all-time numbers to see which streets got safer. 

In [1]:
import pytz
import folium
import pandas as pd
from datetime import datetime, timedelta

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

In [3]:
start_date = pytz.timezone('America/Denver').localize(datetime(2020,7,1))
end_date = cda.most_recent_crash_timestamp()

all_time = cda.street_metrics(start_date, end_date)

In [4]:
start_date = pytz.timezone('America/Denver').localize(datetime(2024,5,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2024,6,1))

last_month = cda.street_metrics(start_date, end_date)

In [5]:
# A street must have at least 100 crashes all-time to be in the comparison
all_time = all_time[all_time.num_crashes >= 100].copy()

In [6]:
all_time

,gid,lrsroute,fullname,street_name_no_cardinal,length_miles,street_line,num_crashes,num_sbi,num_fatality,num_sbi_or_fatality,sum_bicycle_ind,sum_pedestrian_ind,days_in_data,days_between_crashes,crashes_per_mile_per_week,pedestrian_ind_per_mile_per_year
2600,3900,SPEERN BD,N SPEER BLVD,SPEER BLVD,2.634723,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",944,31,7,38,15.0,27.0,1485,1.573093,1.688918,2.520534
3148,1452,ESPEERN BD,E SPEER BLVD,SPEER BLVD,1.060124,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",289,8,2,10,5.0,6.0,1485,5.138408,1.285029,1.392062
479,5587,PARK AV,PARK AVE W,PARK AVE W,1.809664,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",492,20,2,22,6.0,19.0,1485,3.018293,1.281559,2.582376
1631,4229,22ND ST,22ND ST,22ND ST,1.079568,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",291,9,2,11,4.0,15.0,1485,5.103093,1.270617,3.417474
3162,4225,20TH ST,20TH ST,20TH ST,1.453330,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",389,11,0,11,8.0,26.0,1485,3.817481,1.261703,4.400205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,1958,MARTINLUTBD,E MARTIN LUTHER KING JR BLVD,MARTIN LUTHER KING JR BLVD,3.328518,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",135,4,1,5,2.0,2.0,1485,11.000000,0.191185,0.147789
2598,291,35TH AV1,E 35TH AVE,35TH AVE,4.421519,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",175,4,1,5,1.0,3.0,1485,8.485714,0.186568,0.166884
1754,7579,WYALE AV1,W YALE AVE,YALE AVE,3.004537,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",109,9,1,10,2.0,2.0,1485,13.623853,0.171010,0.163725
671,7323,WLOUISIANAV1,W LOUISIANA AVE,LOUISIANA AVE,3.003244,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",103,2,0,2,1.0,3.0,1485,14.417476,0.161666,0.245694


In [7]:
comparison = pd.merge(
    all_time
    , last_month
    , how='inner'
    , on=['gid', 'lrsroute', 'fullname', 'length_miles','street_line']
    , suffixes=['_all_time', '_last_month']
)

In [8]:
comparison['diff_pmpw'] = (
    comparison['crashes_per_mile_per_week_last_month']
    - comparison['crashes_per_mile_per_week_all_time']
)

In [9]:
comparison['expected_crashes'] = (
    comparison['length_miles']
    * comparison['crashes_per_mile_per_week_all_time']
    * (comparison['days_in_data_last_month'] / 7)
)

comparison['diff_expected_crashes'] = comparison['num_crashes_last_month'] - comparison['expected_crashes']

## Difference in Crash Rate per Mile

In [10]:
this_map = folium.Map(prefer_canvas=True, tiles='CartoDB Positron')

good_streets = comparison.sort_values(by='diff_pmpw').head(5)['street_line']
bad_streets = comparison.sort_values(by='diff_pmpw').tail(5)['street_line']

for good_street in good_streets:
    folium.GeoJson(good_street, style_function=lambda x: {'color': 'green'}).add_to(this_map)

for bad_street in bad_streets:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [11]:
comparison.sort_values(by='diff_pmpw')[[
    'gid'
    , 'fullname'
    , 'length_miles'
    , 'crashes_per_mile_per_week_all_time'
    , 'crashes_per_mile_per_week_last_month'
    , 'diff_pmpw'
    , 'num_crashes_all_time'
    , 'num_crashes_last_month'
    , 'expected_crashes'
    , 'diff_expected_crashes'
]]

,gid,fullname,length_miles,crashes_per_mile_per_week_all_time,crashes_per_mile_per_week_last_month,diff_pmpw,num_crashes_all_time,num_crashes_last_month,expected_crashes,diff_expected_crashes
10,3811,WAZEE ST,0.624810,1.139202,0.361400,-0.777801,151,1,3.152189,-2.152189
1,1452,E SPEER BLVD,1.060124,1.285029,0.639000,-0.646029,289,3,6.032997,-3.032997
4,4225,20TH ST,1.453330,1.261703,0.776859,-0.484844,389,5,8.120539,-3.120539
25,2941,N TOWER RD,1.027545,0.889964,0.439507,-0.450458,194,2,4.049832,-2.049832
44,4220,18TH ST,0.843447,0.687415,0.267719,-0.419696,123,1,2.567677,-1.567677
...,...,...,...,...,...,...,...,...,...,...
43,1799,N JOSEPHINE ST,1.876373,0.698389,1.203420,0.505031,278,10,5.803367,4.196633
34,4552,LEETSDALE DR,2.002250,0.814572,1.353316,0.538744,346,12,7.222896,4.777104
35,747,E ALBROOK DR,1.031028,0.772659,1.314065,0.541407,169,6,3.527946,2.472054
82,6862,MORRISON RD,1.486422,0.494714,1.063389,0.568675,156,7,3.256566,3.743434


## Difference in Number of Expected Crashes

In [12]:
this_map = folium.Map(prefer_canvas=True, tiles='CartoDB Positron')

good_streets = comparison.sort_values(by='diff_expected_crashes').head(5)['street_line']
bad_streets = comparison.sort_values(by='diff_expected_crashes').tail(5)['street_line']

for good_street in good_streets:
    folium.GeoJson(good_street, style_function=lambda x: {'color': 'green'}).add_to(this_map)

for bad_street in bad_streets:
    folium.GeoJson(bad_street, style_function=lambda x: {'color': 'red'}).add_to(this_map)
    
this_map.fit_bounds(this_map.get_bounds())
this_map

In [13]:
comparison.sort_values(by='diff_expected_crashes')[[
    'gid'
    , 'fullname'
    , 'length_miles'
    , 'crashes_per_mile_per_week_all_time'
    , 'crashes_per_mile_per_week_last_month'
    , 'diff_pmpw'
    , 'expected_crashes'
    , 'num_crashes_last_month'
    , 'diff_expected_crashes'
]]

,gid,fullname,length_miles,crashes_per_mile_per_week_all_time,crashes_per_mile_per_week_last_month,diff_pmpw,expected_crashes,num_crashes_last_month,diff_expected_crashes
98,1430,GREEN VALLEY RANCH BLVD,4.075550,0.437197,0.110810,-0.326387,7.890909,2,-5.890909
109,920,N CENTRAL PARK BLVD,4.490628,0.406233,0.201136,-0.205098,8.078788,4,-4.078788
107,1431,GREEN VALLEY RANCH BLVD,2.578838,0.414929,0.087561,-0.327367,4.738721,1,-3.738721
21,7584,S FEDERAL BLVD,4.373528,0.950623,0.774454,-0.176169,18.412121,15,-3.412121
120,2938,N TOWER RD,7.211247,0.351677,0.250505,-0.101172,11.230976,8,-3.230976
...,...,...,...,...,...,...,...,...,...
11,5384,W COLFAX AVE,2.317662,1.104387,1.558857,0.454470,11.335354,16,4.664646
34,4552,LEETSDALE DR,2.002250,0.814572,1.353316,0.538744,7.222896,12,4.777104
63,2474,N QUEBEC ST,5.785164,0.562218,0.819672,0.257453,14.404040,21,6.595960
26,4689,N FEDERAL BLVD,5.367276,0.887910,1.220058,0.332148,21.105051,29,7.894949


In [14]:
comparison = comparison.sort_values(by='diff_expected_crashes')

comparison.loc[comparison['diff_expected_crashes'].rank() <= 5, 'Leaderboard'] = (
    comparison.loc[comparison['diff_expected_crashes'].rank() <= 5].apply(
        lambda row: f'{row.diff_expected_crashes:.0f} crashes'
        , axis=1
    )
)

comparison.loc[comparison['diff_expected_crashes'].rank(ascending=False) <= 5, 'Leaderboard'] = (
    comparison.loc[comparison['diff_expected_crashes'].rank(ascending=False) <= 5].apply(
        lambda row: f'+{row.diff_expected_crashes:.0f} crashes'
        , axis=1
    )
)

# comparison['Expected Crashes'] = comparison.expected_c

In [15]:
comparison['street_name'] = comparison['fullname'].str.title()
comparison.loc[
    comparison.Leaderboard.notnull()
    , ['street_name', 'expected_crashes', 'num_crashes_last_month', 'Leaderboard']
].style.format('{:.1f}', subset='expected_crashes')

,street_name,expected_crashes,num_crashes_last_month,Leaderboard
98,Green Valley Ranch Blvd,7.9,2,-6 crashes
109,N Central Park Blvd,8.1,4,-4 crashes
107,Green Valley Ranch Blvd,4.7,1,-4 crashes
21,S Federal Blvd,18.4,15,-3 crashes
120,N Tower Rd,11.2,8,-3 crashes
11,W Colfax Ave,11.3,16,+5 crashes
34,Leetsdale Dr,7.2,12,+5 crashes
63,N Quebec St,14.4,21,+7 crashes
26,N Federal Blvd,21.1,29,+8 crashes
5,E Colfax Ave,30.5,42,+11 crashes
